https://www.kaggle.com/c/cs5785-fall19-final/

# Create an Ensemble Model

#### Get the Descriptions and Tags from File

In [50]:
import os
import numpy as np
import gensim
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler

In [51]:
def parse_descriptions(data_dir, num_doc):
    docs = []
    for i in range(num_doc):
        path = os.path.join(data_dir, "%d.txt" % i)
        with open(path) as f:
            docs.append(f.read())
    return docs

In [121]:
train_tags = parse_descriptions("cs5785-fall19-final/tags_train", num_doc=10000)
test_tags = parse_descriptions("cs5785-fall19-final/tags_test", num_doc=2000)

In [122]:
def parse_tags(tags):
    result = []
    for doc in tags:
        doc = doc.strip('\n').split('\n')
        cat_it = []
        if doc[0] == '':
            result.append('no tag')
        else:
            for tag in doc:
                split_tag = tag.split(':')
                cat_it.append(split_tag[0])
                cat_it.append(split_tag[1])
            parsed = (' ').join(list(cat_it))
            result.append(parsed)
    return np.array(result)

In [123]:
train_tags_parsed = parse_tags(train_tags)
test_tags_parsed = parse_tags(test_tags)

In [144]:
# function to preprocess data
def preprocessing(data):
    stop_words = set(stopwords.words('english')) # find stop words in English language
    lemmatizer = WordNetLemmatizer() # declare nltk lemmatizer

    # iterate through every sentence and replace it by itself lemmatized, without punctuation and without stop words
    for i in range(len(data)):
        sentence_no_punct = ''
        # remove punctuation
        
        for char in data[i]:
            if char not in string.punctuation:
                sentence_no_punct = sentence_no_punct + char
        data[i] = sentence_no_punct

        word_tokens = word_tokenize(data[i])
    
        # remove stop words and lemmatize
        word_tokens = [lemmatizer.lemmatize(word) for word in word_tokens if word not in stop_words and len(word) > 1]
        word_tokens = [lemmatizer.lemmatize(word, 'v') for word in word_tokens]
        word_tokens = [lemmatizer.lemmatize(word, 'a') for word in word_tokens]
        
        # remove conjunction words
        word_tokens = [word for word in word_tokens if word[-2:] != 'nt']
        (data[i]) = ' '.join(word_tokens)
        
    return data

In [145]:
train_descs = parse_descriptions('cs5785-fall19-final/descriptions_train', 10000)
test_descs  = parse_descriptions('cs5785-fall19-final/descriptions_test', 2000)
train_descs = preprocessing(train_descs)
test_descs  = preprocessing(test_descs)

#### Create the Bag of Words

In [57]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format("../GoogleNews-vectors-negative300.bin.gz", binary=True)

In [146]:
def doc_to_vec(sentence, word2vec):
    # get list of word vectors in sentence
    word_vecs = [word2vec.get_vector(w) for w in sentence.split() if w in word2vec.vocab]
    # return average
    return np.stack(word_vecs).mean(0)

In [215]:
x_train_descs = np.array([doc_to_vec(train_descs[i], word2vec) for i in range(len(train_descs))])
x_test = np.array([doc_to_vec(d, word2vec) for d in test_descs])

x_train_tags = np.array([doc_to_vec(train_tags_parsed[i], word2vec) for i in range(len(train_tags_parsed))])
#x_test_tags = np.array([doc_to_vec(d, word2vec) for d in test_tags_parsed])

In [216]:
for i in range(len(x_train_descs)):
    x_train[i] = x_train_descs[i] + x_train_tags[i]
    
#for i in range(len(x_test_descs)): 
    #x_test[i] = x_test_descs[i] + x_test_tags[i]

In [208]:
print(x_train.shape)
print(x_test.shape)

(10000, 300)
(2000, 300)


#### Get Image Feature Vectors

In [217]:
train_feat_sort = pd.read_csv("sorted_train_features.csv", header = None, index_col=None)
test_feat_sort = pd.read_csv("sorted_test_features.csv", header = None, index_col=None)

scalar = StandardScaler()

train_feat_sort_norm = scalar.fit_transform(train_feat_sort)
test_feat_sort_norm = scalar.fit_transform(test_feat_sort)

#train_inter_feat = pd.read_csv("cs5785-fall19-final/features_train/features_resnet1000intermediate_train.csv", header = None, index_col=None)
#test_inter_feat = pd.read_csv("cs5785-fall19-final/features_test/features_resnet1000intermediate_test.csv", header = None, index_col=None)

In [218]:
#train_feat_2 = train_feat

#for i in range(1000):
    #train_feat[0][i] = train_feat[0][i].replace("images_train/", "").replace(".jpg", "")
    #train_feat[0][i] = temp

#train_feat_sort = train_feat.sort_values(by=0)

#for i in range(len(test_feat[0])):
 #   test_feat[0][i] = int(test_feat[0][i].replace("images_test/", "").replace(".jpg", ""))

#test_feat_sort = test_feat.sort_values(by=0)

In [219]:
#train_feat_sort = train_feat.sort_values(by=0)

In [220]:
#for i in range(len(test_feat[0])):
  #  test_feat[0][i] = int(test_feat[0][i].replace("images_test/", "").replace(".jpg", ""))

In [221]:
#y_train = train_feat_sort.drop(columns=0).to_numpy()
#y_test = test_feat_sort.drop(columns=0).to_numpy()

In [222]:
#y_train = train_feat_sort.to_numpy()
#y_test = test_feat_sort.to_numpy()
y_train = train_feat_sort_norm
y_test = test_feat_sort_norm
#y_train_inter = train_inter_feat_sort.drop(columns=0).to_numpy()
#y_test_inter = test_inter_feat_sort.drop(columns=0).to_numpy()

#### Create A Train/Test Split

In [223]:
import random
num_train = 8000
num_dev = 2000
num_test = 2000
split = list(range(num_train + num_dev))
random.shuffle(split)

In [234]:
x_train_dev = x_train[split[:num_train]]
y_train_dev = y_train[split[:num_train]]
x_test_dev = x_train_descs[split[num_train:]]
y_test_dev = y_train[split[num_train:]]

train_tags_dev = train_tags_parsed[split[:num_train]]
test_tags_dev = train_tags_parsed[split[num_train:]]

#### For Later when we need the tags of the whole train and test set

In [235]:
tag_train = np.array([doc_to_vec(d, word2vec) for d in train_tags_parsed])
tag_test = np.array([doc_to_vec(d, word2vec) for d in test_tags_parsed])

#### For Development Right Now when we need the smaller train and test set to check accuracy

In [236]:
train_tags_dev = np.array([doc_to_vec(d, word2vec) for d in train_tags_dev])
test_tags_dev = np.array([doc_to_vec(d, word2vec) for d in test_tags_dev])

### Relate Descriptions and Tags with a Model --- SVM

In [143]:
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
supvm = MultiOutputRegressor(SVR(gamma = 'scale')).fit(x_train_dev, train_tags_dev)
tag_preds = supvm.predict(x_test_dev)

KeyboardInterrupt: 

### Relate Image Features and Descriptions Model

In [ ]:
from sklearn.neural_network import MLPRegressor

clf = MLPRegressor(solver='sgd', alpha=1e-5, hidden_layer_sizes=(300,), random_state=1, max_iter = 1000)
clf.fit(x_train_dev, y_train_dev)
image_preds = clf.predict(x_test_dev)

In [174]:
# for tags
from sklearn.neural_network import MLPRegressor

clf = MLPRegressor(solver='sgd', alpha=1e-5, hidden_layer_sizes=(300,), random_state=1, max_iter = 1000)
clf.fit(x_train_dev, train_tags_dev)
tag_preds = clf.predict(x_test_dev)

In [179]:
from sklearn.kernel_ridge import KernelRidge

kr = KernelRidge()
kr.fit(x_train_dev, train_tags_dev)
tag_preds = kr.predict(x_test_dev)

### Lasso Regressor with Cross-Validation to relate descriptors and tags (maybe try instead of mlp too?)

In [233]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV

lasso = Lasso()
parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]}
lasso_regressor = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv = 5)
lasso_regressor.fit(x_train, tag_train)
tag_preds = lasso_regressor.predict(x_test)

print(lasso_regressor.best_params_)
print(lasso_regressor.best_score_)

C:\Users\evatr\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.8129782676696777, tolerance: 0.009223545901477337
  positive)


KeyboardInterrupt: 

Best lasso params: {'alpha': 1e-08}
-0.0024774808902293445

### Ridge Regressor with Cross-Validation 

In [237]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
ridge = Ridge()
parameters = {'alpha': [1e-15, 1e-10, 1e-8, 1e-4, 1e-3, 1e-2, 1, 5, 10, 20]}

ridge_regressor = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv = 5)
ridge_regressor.fit(x_train_dev, train_tags_dev)
tag_preds = ridge_regressor.predict(x_test_dev)

print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

{'alpha': 0.01}
-0.00019588022842071952


In [163]:
print(tag_preds.shape)
print(test_tags_dev.shape)
print(y_train_dev.shape)
print(x_train_dev.shape)

(2000, 300)
(2000, 300)
(8000, 1000)
(8000, 300)


Best Ridge parameters: {'alpha': 1}
-0.0024467481765896083

### Combine the Models

#### Try Evaluating the Nearest Neighbors and Take A New Ranking Based on Both

In [164]:
# function to return the number of nearest neighbors specified
def knn_function(data, point, neighbors_number):
    knn = NearestNeighbors(n_neighbors=neighbors_number)
    knn.fit(data)
    return knn.kneighbors(point, neighbors_number, return_distance=False)

In [165]:
tag_neighbors = knn_function(test_tags_dev, tag_preds, 50)
#image_neighbors = knn_function(y_test_dev, image_preds, 50)

In [166]:
def combine_tags_images(ims, tags):
    neighbors = [[] for _ in range(len(ims))]
    
    for i in range(len(ims)):
        neighbors[i] += [im for im in ims[i] if im in tags]
        #neighbors[i] += [im for im in tags[i]]
        
        for im in neighbors[i]:
            if len(neighbors[i]) == 20:
                break
            if not im in neighbors[i]:
                neighbors[i] += [im]
        neighbors[i] = np.array(neighbors[i])

    return np.array(neighbors)

In [ ]:
#neighbors1 = []
#for i in range(len(tag_neighbors)):
 #   neighbors1.append(np.array(list(set(tag_neighbors[i]) & set(image_neighbors[i]))[:20]))
neighbors1 = combine_tags_images(image_neighbors, tag_neighbors)

In [ ]:
neighbors1 = np.array(neighbors1)

### Evaluate the Model

In [167]:
def scores(i):
    return (20+1-(i+1))/20

In [157]:
def eval_accuracy(ytest, neighbors):
    # EVALUATE THE MODEL USING THE MEAN AVERAGE PRECISION AT 20
    scs = []
    for i in range(len(neighbors)):
        good = False
        for j, n in enumerate(neighbors[i]):
            if i == n:
                scs.append(scores(j))
                good = True
                break
        if good == False:
            scs.append(0)
    return sum(scs)/len(neighbors)

In [158]:
#print('accuracy of svm tags', eval_accuracy(y_test_dev, tag_neighbors[:20]))

In [168]:
print('accuracy of svm tags', eval_accuracy(y_test_dev, tag_neighbors[:20]))

accuracy of svm tags 0.2875000000000001


In [160]:
print('accuracy of image features', eval_accuracy(y_test_dev, image_neighbors[:20]))

NameError: name 'image_neighbors' is not defined

In [ ]:
#print('accuracy of svm tags combined with image features', eval_accuracy(y_test_dev, neighbors1))

In [ ]:
print('accuracy of svm tags combined with image features', eval_accuracy(y_test_dev, neighbors1))

In [ ]:
neighbors_adjusted = []
file_list = []
for i, row in enumerate(tag_neighbors):
    newlst = []
    for val in row:
        newlst.append(str(val)+'.jpg')
    neighbors_adjusted.append((' ').join(newlst))
    #file_list.append(str(i)+'.txt')

In [136]:
import csv
def outputCSV(predictions):
    with open("image_prediction_svm-mlp.csv", "w") as outputFile:
        headers = ["Descritpion_ID", "Top_20_Image_IDs"]
        fileWriter = csv.DictWriter(outputFile, fieldnames=headers)
        fileWriter.writeheader()
        for index, pred in enumerate(predictions):
            fileWriter.writerow({headers[0]: "{}.txt".format(index), headers[1]: ''.join(predictions[index])})

In [137]:
outputCSV(neighbors_adjusted)

In [238]:
from scipy.spatial.distance import cdist

def dist_matrix(x1, x2):
    return cdist(x1,x2,'cosine')

In [239]:
val_distances = dist_matrix(tag_preds, test_tags_dev)

In [ ]:
val_distances_2 = dist_matrix(image_preds, y_test_dev)

In [240]:
val_scores = []
val_pos_list = []

for i in range(len(test_tags_dev)):
    pred_dist_idx = list(np.argsort(val_distances[i]))
    val_pos = pred_dist_idx.index(i)
    val_pos_list.append(val_pos)
    if val_pos < 20:
        val_scores.append(1 / (val_pos + 1))
    else:
        val_scores.append(0.0)

print("Development MAP@20:", np.mean(val_scores))
print("Mean index of true image", np.mean(val_pos_list))
print("Median index of true image", np.median(val_pos_list))

Development MAP@20: 0.01583913900212275
Mean index of true image 445.482
Median index of true image 279.0


In [ ]:
val_scores = []
val_pos_list = []

for i in range(len(y_test_dev)):
    pred_dist_idx = list(np.argsort(val_distances_2[i]))
    val_pos = pred_dist_idx.index(i)
    val_pos_list.append(val_pos)
    if val_pos < 20:
        val_scores.append(1 / (val_pos + 1))
    else:
        val_scores.append(0.0)

print("Development MAP@20:", np.mean(val_scores))
print("Mean index of true image", np.mean(val_pos_list))
print("Median index of true image", np.median(val_pos_list))